In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import cupy as cp
#from scipy import signal
#from scipy import ndimage
#from scipy.signal import fftconvolve, oaconvolve
#from cupyx.scipy.ndimage import filters

from scipy.ndimage import convolve as conv
from scipy.signal import convolve2d, fftconvolve, oaconvolve
#from cupyx.scipy.ndimage.filters import convolve as cupy_conv
from cupyx.scipy.ndimage import convolve as cupy_conv

# Comparison

In [14]:
a = np.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = np.array([[1,1,1,1,1,1],[1,1,1,1,0,1],[1,1,1,0,0,1],[1,1,0,0,0,1],[1,0,0,0,0,1]])
print(conv(a, k, mode='constant',origin=(0,-1)))
print(convolve2d(a,k, mode='full'))
print(convolve2d(a,k, mode='same'))

[[11 21 14 17]
 [22 26 23 29]
 [15 23 14 22]
 [12 14  7 14]]
[[ 1  3  3  3  3  3  2  0  0]
 [ 6 11 11 15 14 13  9  4  4]
 [ 6 11 11 21 14 17 16  7 11]
 [15 23 22 26 23 29 15  0 11]
 [15 22 15 23 14 22  8  0 11]
 [14 15 12 14  7 14  6  0 11]
 [ 9 12  3  7  0  9  3  0  7]
 [ 9  3  0  0  0  9  3  0  0]]
[[11 21 14 17]
 [22 26 23 29]
 [15 23 14 22]
 [12 14  7 14]]


In [29]:
%%timeit
a=cp.linspace(0,1000,500)

36.7 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [30]:
%%timeit
a = cp.arange(0,1000,2)

27.9 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [2]:
# Compare results from five different convolution methods

# 4 CPU methods (1 ndimage method + 3 filter methods)
a = np.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = np.array([[1,1,1,1,1],[1,1,1,1,0],[1,1,1,0,0],[1,1,0,0,0],[1,0,0,0,0]])
print(conv(a, k, mode='constant'))
print(convolve2d(a,k, mode='same'))
print(fftconvolve(a,k, mode='same'))
print(oaconvolve(a,k, mode='same'))

# 1 GPU method (ndimage method)
a = cp.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = cp.array([[1,1,1,1,1],[1,1,1,1,0],[1,1,1,0,0],[1,1,0,0,0],[1,0,0,0,0]])
print(cupy_conv(a, k, mode='constant'))

[[11 21 14 11]
 [22 26 23 14]
 [15 23 14  7]
 [12 14  7  0]]
[[11 21 14 11]
 [22 26 23 14]
 [15 23 14  7]
 [12 14  7  0]]
[[1.10000000e+01 2.10000000e+01 1.40000000e+01 1.10000000e+01]
 [2.20000000e+01 2.60000000e+01 2.30000000e+01 1.40000000e+01]
 [1.50000000e+01 2.30000000e+01 1.40000000e+01 7.00000000e+00]
 [1.20000000e+01 1.40000000e+01 7.00000000e+00 1.77635684e-15]]
[[1.10000000e+01 2.10000000e+01 1.40000000e+01 1.10000000e+01]
 [2.20000000e+01 2.60000000e+01 2.30000000e+01 1.40000000e+01]
 [1.50000000e+01 2.30000000e+01 1.40000000e+01 7.00000000e+00]
 [1.20000000e+01 1.40000000e+01 7.00000000e+00 1.77635684e-15]]
[[11 21 14 11]
 [22 26 23 14]
 [15 23 14  7]
 [12 14  7  0]]


# Matrix center specification

Depending on the shape of the kernal, the "origin" of the ndimage methods needs to be set properly to match the results with the signal methods 

In [30]:
# k.shape = (even,even), origin = (-1,-1) required
a = np.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = np.array([[1,1,1,1,1,1],[1,1,1,1,1,0],[1,1,1,1,0,0],[1,1,1,0,0,0],[1,1,0,0,0,0],[1,0,0,0,0,0]])
origin = (-1,-1)
print(conv(a, k, mode='constant', origin = origin))
print(convolve2d(a,k, mode='same'))

[[11 22 21 14]
 [23 33 26 23]
 [22 26 23 14]
 [15 23 14  7]]
[[11 22 21 14]
 [23 33 26 23]
 [22 26 23 14]
 [15 23 14  7]]


In [17]:
# k.shape = (odd,even), origin = (0,-1) required
a = np.array([[1, 2, 0, 0,1],[5, 3, 0, 4,1],[0, 0, 0, 7,1],[9, 3, 0, 0,1]])
k = np.array([[1,1,1,1,1,1],[1,1,1,1,0,1],[1,1,1,0,0,1],[1,1,0,0,0,1],[1,0,0,0,0,1]])
print(conv(a, k, mode='constant',origin=(0,-1)))
print(convolve2d(a,k, mode='same'))

[[11 21 17 20 19]
 [22 26 27 33 18]
 [15 23 18 25 10]
 [12 14 10 16  7]]
[[11 21 17 20 19]
 [22 26 27 33 18]
 [15 23 18 25 10]
 [12 14 10 16  7]]


In [21]:
# k.shape = (even,odd), origin = (-1,0) required
a = np.array([[1, 2, 0, 0,1],[5, 3, 0, 4,1],[0, 0, 0, 7,1],[9, 3, 0, 0,1]])
k = np.array([[1,1,1,1,1],[1,1,1,1,0],[1,1,1,0,0],[1,1,0,0,0],[1,0,0,0,0],[0,0,0,0,0]])
print(conv(a, k, mode='constant',origin=(-1,0)))
print(convolve2d(a,k, mode='same'))

[[11 21 17 14 14]
 [22 26 27 18 10]
 [15 23 18 10  2]
 [12 14 10  2  1]]
[[11 21 17 14 14]
 [22 26 27 18 10]
 [15 23 18 10  2]
 [12 14 10  2  1]]


# Timing

In [12]:
# Compare timings with different matrix size N 
# N > 100 can take significant time
N = 10
a = np.random.randint(5, size=(N, N))
k = np.random.randint(5, size=(2*N, 2*N))
%timeit -n 3 -r 4 conv(a, k, mode='constant', origin = (-1,-1))
%timeit -n 3 -r 4 convolve2d(a, k, mode='same')
%timeit -n 3 -r 4 fftconvolve(a, k, mode='same')
%timeit -n 3 -r 4 oaconvolve(a, k, mode='same')

a = cp.random.randint(5, size=(N, N))
k = cp.random.randint(5, size=(2*N, 2*N))
%timeit -n 3 -r 4 cupy_conv(a, k, mode='constant', origin = (-1,-1))

1.29 ms ± 431 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)
137 µs ± 850 ns per loop (mean ± std. dev. of 4 runs, 3 loops each)
337 µs ± 127 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)
275 µs ± 53.9 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)
88.7 µs ± 16.5 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)


In [14]:
# Check the time for array assignments
N = 100
%timeit -n 3 -r 4 a = np.random.randint(5, size=(N, N))
%timeit -n 3 -r 4 a = cp.random.randint(5, size=(N, N))

496 µs ± 159 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)
1.28 ms ± 211 µs per loop (mean ± std. dev. of 4 runs, 3 loops each)


In [ ]:
# Not sure if this function is required to time GPU commands
cupy.cuda.runtime.deviceSynchronize() 